In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

In [2]:
np.random.seed(42)

n_orders = 10000
start_date = datetime(2024,1,1)

order_ids = [f"ORD{i}" for i in range(1, n_orders+1)]

orders = pd.DataFrame({
    "order_id": order_ids,
    "order_date": [start_date + timedelta(days=random.randint(0,180)) for _ in range(n_orders)],
    "customer_id": np.random.randint(1000, 6000, n_orders),
    "order_value": np.round(np.random.uniform(10,300,n_orders),2),
    "payment_type": np.random.choice(["COD","UPI","Credit Card","Wallet"], n_orders),
    "warehouse": np.random.choice(["Mumbai","Delhi","Bangalore","Hyderabad"], n_orders),
    "order_status": np.random.choice(["Delivered","Delayed","Cancelled"], n_orders, p=[0.75,0.2,0.05])
})

In [3]:
fulfillment = []

for _, row in orders.iterrows():
    order_time = row["order_date"]

    pick = order_time + timedelta(hours=random.randint(1,6))
    pack = pick + timedelta(hours=random.randint(1,4))
    ship = pack + timedelta(hours=random.randint(2,10))

    if row["order_status"] == "Delayed":
        delivery = ship + timedelta(days=random.randint(4,7))
    else:
        delivery = ship + timedelta(days=random.randint(1,3))

    fulfillment.append([row["order_id"], pick, pack, ship, delivery, random.choice(["BlueDart","Delhivery","Ecom Express"])])

fulfillment = pd.DataFrame(fulfillment, columns=["order_id","pick_time","pack_time","ship_time","delivery_time","carrier"])

In [4]:
skus = [f"SKU{i}" for i in range(1,101)]

inventory = pd.DataFrame({
    "sku": skus,
    "warehouse": np.random.choice(["Mumbai","Delhi","Bangalore","Hyderabad"],100),
    "stock": np.random.randint(0,500,100),
    "reorder_level": np.random.randint(50,150,100),
    "lead_time_days": np.random.randint(3,10,100)
})

In [5]:
tickets = []

for i in range(3000):
    order = random.choice(order_ids)
    created = start_date + timedelta(days=random.randint(0,180))
    resolved = created + timedelta(hours=random.randint(2,72))

    tickets.append([f"TKT{i}", order, random.choice(["Late Delivery","Damaged Item","Refund","Wrong Item"]),
                    created, resolved, "Closed"])

tickets = pd.DataFrame(tickets, columns=["ticket_id","order_id","issue_type","created_time","resolved_time","status"])


In [7]:
orders.to_csv("oms_orders.csv", index=False)
fulfillment.to_csv("wms_fulfillment.csv", index=False)
inventory.to_csv("wms_inventory.csv", index=False)
tickets.to_csv("support_tickets.csv", index=False)


In [8]:
import pandas as pd

orders = pd.read_csv("oms_orders.csv", parse_dates=["order_date"])
fulfillment = pd.read_csv("wms_fulfillment.csv", parse_dates=["pick_time","pack_time","ship_time","delivery_time"])
tickets = pd.read_csv("support_tickets.csv", parse_dates=["created_time","resolved_time"])

df = orders.merge(fulfillment, on="order_id", how="left")
df = df.merge(tickets, on="order_id", how="left")


In [9]:
df["pick_delay_hrs"] = (df["pick_time"] - df["order_date"]).dt.total_seconds()/3600
df["pack_delay_hrs"] = (df["pack_time"] - df["pick_time"]).dt.total_seconds()/3600
df["ship_delay_hrs"] = (df["ship_time"] - df["pack_time"]).dt.total_seconds()/3600
df["delivery_days"] = (df["delivery_time"] - df["ship_time"]).dt.days

df["support_resolution_hrs"] = (df["resolved_time"] - df["created_time"]).dt.total_seconds()/3600


In [11]:
df.groupby("warehouse")["delivery_days"].mean().sort_values(ascending=False)

warehouse
Delhi        2.705361
Hyderabad    2.667454
Mumbai       2.663717
Bangalore    2.659882
Name: delivery_days, dtype: float64

In [12]:
df.groupby("warehouse")["pick_delay_hrs"].mean().sort_values(ascending=False)

warehouse
Delhi        3.512534
Mumbai       3.508657
Bangalore    3.500370
Hyderabad    3.455333
Name: pick_delay_hrs, dtype: float64

In [13]:
df.groupby("issue_type")["support_resolution_hrs"].mean().sort_values(ascending=False)

issue_type
Damaged Item     38.335072
Wrong Item       36.511719
Refund           36.491547
Late Delivery    35.544540
Name: support_resolution_hrs, dtype: float64

In [14]:
df["sla_breached"] = df["delivery_days"] > 3
df.groupby("warehouse")["sla_breached"].mean() * 100

warehouse
Bangalore    19.319023
Delhi        19.861165
Hyderabad    19.362456
Mumbai       19.161216
Name: sla_breached, dtype: float64

In [15]:
inventory = pd.read_csv("wms_inventory.csv")

In [16]:
inventory["stockout_risk"] = inventory["stock"] < inventory["reorder_level"]

In [17]:
critical = inventory[inventory["stockout_risk"] == True]
critical.head()

,sku,warehouse,stock,reorder_level,lead_time_days,stockout_risk
11,SKU12,Hyderabad,44,121,7,True
12,SKU13,Bangalore,0,146,8,True
17,SKU18,Delhi,55,112,6,True
21,SKU22,Mumbai,6,142,4,True
22,SKU23,Mumbai,19,66,8,True


In [18]:
inventory.groupby("warehouse")["stockout_risk"].mean() * 100

warehouse
Bangalore    12.500000
Delhi        15.384615
Hyderabad    12.000000
Mumbai       20.000000
Name: stockout_risk, dtype: float64

In [19]:
inventory["reorder_priority"] = (inventory["reorder_level"] - inventory["stock"]) * inventory["lead_time_days"]
inventory.sort_values("reorder_priority", ascending=False).head(10)

,sku,warehouse,stock,reorder_level,lead_time_days,stockout_risk,reorder_priority
12,SKU13,Bangalore,0,146,8,True,1168
39,SKU40,Bangalore,23,138,7,True,805
29,SKU30,Mumbai,0,94,7,True,658
21,SKU22,Mumbai,6,142,4,True,544
11,SKU12,Hyderabad,44,121,7,True,539
44,SKU45,Mumbai,56,115,8,True,472
78,SKU79,Mumbai,17,110,5,True,465
22,SKU23,Mumbai,19,66,8,True,376
17,SKU18,Delhi,55,112,6,True,342
88,SKU89,Delhi,85,139,6,True,324


In [20]:
df["support_resolution_hrs"].mean()

np.float64(36.748333333333335)

In [21]:
df.groupby("issue_type")["support_resolution_hrs"].mean().sort_values(ascending=False)

issue_type
Damaged Item     38.335072
Wrong Item       36.511719
Refund           36.491547
Late Delivery    35.544540
Name: support_resolution_hrs, dtype: float64

In [22]:
df["optimized_resolution"] = df["support_resolution_hrs"]

df.loc[df["issue_type"]=="Late Delivery", "optimized_resolution"] *= 0.7
df.loc[df["issue_type"]=="Refund", "optimized_resolution"] *= 0.6

In [23]:
before = df["support_resolution_hrs"].mean()
after = df["optimized_resolution"].mean()

improvement = (before - after) / before * 100

before, after, improvement


(np.float64(36.748333333333335),
 np.float64(30.532833333333333),
 np.float64(16.913692230940182))

In [24]:
df.to_csv("ecommerce_ops_analytics.csv", index=False)
inventory.to_csv("inventory_analytics.csv", index=False)